In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 15)

LARGE_FIGSIZE = (12, 8)

matplotlib = 3.3.2

numpy = 1.19.2

pandas = 1.1.3

psycopg2 = 2.8.5 

python = 3.8.5

In [2]:
df = pd.read_csv('C:/Users/Kate/PycharmProjects/test/test/P9-ConsumerComplaints.csv',
                 header = 0,
                 na_values=['N/A'],
                 index_col = 'Complaint ID',
                 true_values=['Yes'],
                 false_values=['No'],
                 # keep_default_na=False,
                 low_memory=False
                 )
df.head()

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed
Complaint ID,,,,,,,,,,,,,,,,,
468882,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,True,False
468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,True,False
468879,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,True,False
468949,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False
475823,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False


In [3]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# df = pd.read_csv('https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P9-ConsumerComplaints.csv')


In [4]:
df['Date Received'] = pd.to_datetime(df['Date Received'])

In [5]:
df.dtypes

Date Received                   datetime64[ns]
Product Name                            object
Sub Product                             object
Issue                                   object
Sub Issue                               object
                                     ...      
Submitted via                           object
Date Sent to Company                    object
Company Response to Consumer            object
Timely Response                           bool
Consumer Disputed                       object
Length: 17, dtype: object

In [6]:
date_start = pd.to_datetime('2013-07-29')
date_end = pd.to_datetime('2013-07-30')

In [7]:
# all product names
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)]['Product Name'].unique()

array(['Consumer Loan', 'Bank account or service', 'Mortgage',
       'Debt collection', 'Credit card', 'Credit reporting',
       'Student loan'], dtype=object)

In [8]:
# complaints for each product
df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end)][['Product Name','Issue']].sort_values('Product Name')

,Product Name,Issue
Complaint ID,,
469537,Bank account or service,Problems caused by my funds being low
470611,Bank account or service,"Account opening, closing, or management"
479508,Bank account or service,Problems caused by my funds being low
469446,Bank account or service,Deposits and withdrawals
470633,Bank account or service,"Account opening, closing, or management"
...,...,...
469340,Student loan,Repaying your loan
467427,Student loan,Repaying your loan
479648,Student loan,Repaying your loan


In [9]:
df['Timely Response'].unique()

array([ True, False])

In [10]:
df['Consumer Disputed'].unique()

array([False, True, nan], dtype=object)

In [19]:
# parse timely response
df.loc[df['Timely Response'] != True, 'TR False NA'] = False  
df.loc[df['Timely Response'] == True, 'TR False NA'] = True

In [12]:
# parse consumer disputed
df.loc[df['Consumer Disputed'] != True, 'CD False NA'] = False  
df.loc[df['Consumer Disputed'] == True, 'CD False NA'] = True

In [104]:
# how much with timely response
df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['TR False NA'] == True)][['Product Name','TR False NA']].groupby('Product Name').count()

,TR False NA
Product Name,
Bank account or service,34
Consumer Loan,10
Credit card,26
Credit reporting,22
Debt collection,25
Mortgage,75
Student loan,7


In [105]:
# how much disputed by customer

df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['CD False NA'] == True)][['Product Name','CD False NA']].groupby('Product Name').count()

,CD False NA
Product Name,
Bank account or service,3
Consumer Loan,3
Credit card,10
Credit reporting,2
Debt collection,3
Mortgage,16


In [106]:
# counting number of timely responses for each issue in the given date range

tr_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['TR False NA'] == True)][['Product Name', 'Issue', 'TR False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()


In [107]:
tr_ic_df.head()

TR False NA
Product Name            Issue                                                
Bank account or service Account opening, closing, or management            12
                        Deposits and withdrawals                           13
                        Making/receiving payments, sending money            2
                        Problems caused by my funds being low               4
                        Using a debit or ATM card                           3

In [108]:
# counting number of consumer disputed for each issue in the given date range

cd_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['CD False NA'] == True)][['Product Name', 'Issue', 'CD False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

In [109]:
cd_ic_df.head()

CD False NA
Product Name            Issue                                               
Bank account or service Account opening, closing, or management            3
Consumer Loan           Managing the loan or lease                         3
Credit card             APR or interest rate                               1
                        Billing disputes                                   2
                        Billing statement                                  1

In [110]:
# full join on index
res = tr_ic_df.merge(cd_ic_df, left_index=True, right_index=True, how='outer').fillna(0)
res['CD False NA'] = res['CD False NA'].astype(int, errors='raise')
res.dtypes

TR False NA    int64
CD False NA    int32
dtype: object

In [111]:
res.head()

TR False NA  \
Product Name            Issue                                                   
Bank account or service Account opening, closing, or management            12   
                        Deposits and withdrawals                           13   
                        Making/receiving payments, sending money            2   
                        Problems caused by my funds being low               4   
                        Using a debit or ATM card                           3   

                                                                  CD False NA  
Product Name            Issue                                                  
Bank account or service Account opening, closing, or management             3  
                        Deposits and withdrawals                            0  
                        Making/receiving payments, sending money            0  
                        Problems caused by my funds being low               0  
                        Using a debit or ATM card                           0

In [112]:
# 
res.sort_values(by = ['Product Name', 'Issue', 'TR False NA', 'CD False NA'], ascending = [True, True, False, False])

TR False NA  \
Product Name            Issue                                                   
Bank account or service Account opening, closing, or management            12   
                        Deposits and withdrawals                           13   
                        Making/receiving payments, sending money            2   
                        Problems caused by my funds being low               4   
                        Using a debit or ATM card                           3   
...                                                                       ...   
Mortgage                Loan servicing, payments, escrow account           21   
                        Other                                               1   
                        Settlement process and costs                        3   
Student loan            Problems when you are unable to pay                 1   
                        Repaying your loan                                  6   

                                                                  CD False NA  
Product Name            Issue                                                  
Bank account or service Account opening, closing, or management             3  
                        Deposits and withdrawals                            0  
                        Making/receiving payments, sending money            0  
                        Problems caused by my funds being low               0  
                        Using a debit or ATM card                           0  
...                                                                       ...  
Mortgage                Loan servicing, payments, escrow account            4  
                        Other                                               0  
                        Settlement process and costs                        1  
Student loan            Problems when you are unable to pay                 0  
                        Repaying your loan                                  0  

[38 rows x 2 columns]

In [113]:

# performance test

# del tr_ic_df, cd_ic_df, res

import time
m_start = time.time()

tr_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['TR False NA'] == True)][['Product Name', 'Issue', 'TR False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

cd_ic_df = df[(df['Date Received'] >= date_start) &\
   (df['Date Received'] <= date_end) &\
   (df['CD False NA'] == True)][['Product Name', 'Issue', 'CD False NA']]\
.groupby(['Product Name', 'Issue'])\
.count()

res = tr_ic_df.merge(cd_ic_df, left_index=True, right_index=True, how='outer').fillna(0)
res['CD False NA'] = res['CD False NA'].astype(int, errors='raise')

res.sort_values(by = ['Product Name', 'Issue', 'TR False NA', 'CD False NA'], ascending = [True, True, False, False])

m_end = time.time()

m_end - m_start

0.032000064849853516

In [125]:
# the second query
# which state has the biggest amount of issues for Encore Capital Group

bad_state = df[df['Company'] == 'Encore Capital Group'][['State Name', 'Issue']]\
.groupby(['State Name'])\
.count()\
.sort_values(by = 'Issue', ascending = False)\
.iloc[0].name

In [127]:
bad_state

'CA'

In [133]:
df[(df['Company'] == 'Encore Capital Group') &\
   (df['State Name'] == bad_state)][['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue']]\
.groupby(['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue'])\
.count()


Empty DataFrame
Columns: []
Index: [(Encore Capital Group, CA, Consumer Loan, Installment loan, Managing the loan or lease), (Encore Capital Group, CA, Consumer Loan, Installment loan, Problems when you are unable to pay), (Encore Capital Group, CA, Consumer Loan, Personal line of credit, Account terms and changes), (Encore Capital Group, CA, Debt collection, Credit card, Communication tactics), (Encore Capital Group, CA, Debt collection, Credit card, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Credit card, Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, Credit card, False statements or representation), (Encore Capital Group, CA, Debt collection, Credit card, Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Credit card, Taking/threatening an illegal action), (Encore Capital Group, CA, Debt collection, I do not know, Communication tactics), (Encore Capital Group, CA, Debt collection, I do not know, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, I do not know, Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, I do not know, False statements or representation), (Encore Capital Group, CA, Debt collection, I do not know, Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Medical, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Mortgage, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Non-federal student loan, Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Communication tactics), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Cont'd attempts collect debt not owed), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Disclosure verification of debt), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), False statements or representation), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Improper contact or sharing of info), (Encore Capital Group, CA, Debt collection, Other (i.e. phone, health club, etc.), Taking/threatening an illegal action)]

[23 rows x 0 columns]

In [134]:
# performance test 2

m_start = time.time()

bad_state = df[df['Company'] == 'Encore Capital Group'][['State Name', 'Issue']]\
.groupby(['State Name'])\
.count()\
.sort_values(by = 'Issue', ascending = False)\
.iloc[0].name

res = df[(df['Company'] == 'Encore Capital Group') &\
   (df['State Name'] == bad_state)][['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue']]\
.groupby(['Company', 'State Name', 'Product Name', 'Sub Product', 'Issue'])\
.count()

m_end = time.time()

m_end - m_start

0.039999961853027344

In [37]:
import psycopg2
connection = psycopg2.connect(user="postgres",
                              password="o0pO)P",
                              host="127.0.0.1",
                              port="5433",
                              database="postgres")

cursor = connection.cursor()

In [38]:
def issuecounter(cursor, date_from, date_to):
    query = """
        select  product_name, issue,
        count(timely_response) filter (where timely_response_bool is true) trc,
        count(consumer_disputed) filter (where consumer_disputed_bool is true) cdc
        from user_complaints
        where date_received >= %s and date_received <= %s
        group by product_name, issue
        order by product_name, issue, count(issue) desc
        
        
    ;"""
    cursor.execute(query, (date_from, date_to))
    header = "product_name, issues, count_timely_response, count_consumer_disputed"
    print(header)
    for row in cursor:
        print(row)

In [39]:
%%timeit
issuecounter(cursor, '2013-07-29', '2013-07-30')
# Postgres 30 ms : 32 ms Pandas

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

In [12]:
def bad_company(cursor, company):
    query = """
    with t1 as (
	select
		company, state_name, count(issue) ci 
	from user_complaints
	where company = 'Encore Capital Group'
	group by company, state_name
	order by ci desc
	limit 1)
    select 
	t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue
    from user_complaints uc, t1
    where uc.company = t1.company and uc.state_name = t1.state_name
    group by t1.company, t1.state_name, uc.product_name, uc.sub_product, uc.issue       
    ;"""
    cursor.execute(query, company)
    header = "company, state_name, product_name, sub_product, issue"
    print(header)
    for row in cursor:
        print(row)

In [13]:
%%timeit
bad_company(cursor, 'Encore Capital Group')

# Postgres 22 ms : 40 ms Pandas

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or representation')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Improper contact or sharing of info')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Taking/threatening an illegal action')
('Encore Capital Group', 'CA', 'Debt coll

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

company, state_name, product_name, sub_product, issue
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Managing the loan or lease')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Installment loan', 'Problems when you are unable to pay')
('Encore Capital Group', 'CA', 'Consumer Loan', 'Personal line of credit', 'Account terms and changes')
('Encore Capital Group', 'CA', 'Credit card', '', 'Customer service / Customer relations')
('Encore Capital Group', 'CA', 'Credit card', '', 'Other')
('Encore Capital Group', 'CA', 'Credit reporting', '', 'Incorrect information on credit report')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Communication tactics')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', "Cont'd attempts collect debt not owed")
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'Disclosure verification of debt')
('Encore Capital Group', 'CA', 'Debt collection', 'Credit card', 'False statements or represen

In [14]:
if (connection):
    cursor.close()
    connection.close()